In [2]:
import investpy as ip
import yahooquery as yq
from pandas_datareader import data as pdr
import pandas as pd
from tqdm import trange
import yfinance as yf


In [6]:
df = pd.read_excel('./datas/MSCIeurope.xlsx')
df.head()

,Ticker,Ticker Yahoo,Nom,Secteur,Equity class,Market Share,Weights,Valeur notionnelle,Actions,ISIN,Currency
0,NESN,NSRGY,NESTLE SA,Biens de consommation de base,Actions,"EUR 48 274 991,14",4.67,48274991.14,420838,CH0038863350,CHF
1,ASML,ASML,ASML HOLDING NV,Technologie de l'informatioon,Actions,"EUR 34 906 428,20",3.38,34906428.20,60802,NL0010273215,EUR
2,ROG,ROG.SW,ROCHE HOLDING PAR AG,Santé,Actions,"EUR 33 188 774,68",3.21,33188774.68,104989,CH0012032048,CHF
3,MC,MC.PA,LVMH,Biens de consommation cycliques,Actions,"EUR 29 028 191,40",2.81,29028191.40,41457,FR0000121014,EUR
4,NOVOB,NVO,NOVO NORDISK CLASS B,Santé,Actions,"EUR 28 242 417,87",2.73,28242417.87,247375,DK0060534915,DKK


## get yahoo ticker

In [2]:
def find_ticker(isin):
    try:
        company_name = ip.stocks.search_stocks(by='isin', value=str(isin))
        company_name = company_name["name"][0].split(' ')[0]
        symbol = yq.search(company_name)["quotes"][0]["symbol"]
    except:
        symbol = None
    return symbol

In [ ]:
for i in range(len(df)):
    df.loc[i, "Ticker Yahoo"] = find_ticker(df.loc[i, "ISIN"])
df = df[['Ticker', 'Ticker Yahoo', 'Nom', 'Secteur', 'Equity class', 'Market Share', 'Weights',
       'Valeur notionnelle', 'Actions', 'ISIN', 'Currency', ]] 

In [10]:
# total weight
print(df["Weights"].sum())
# lost weights
print(df.loc[(df['Ticker Yahoo'].isna()), 'Weights'].sum())

99.98999999999998
6.82


### get ESG scores

In [7]:
esg_scores = pd.DataFrame(columns = ['Ticker Yahoo', 'Environment Score', 'Social Score', 'Governance Score', 'Total Score'], index = range(len(df)))
# trange = tqdm(range(len(df)))

for i in trange(len(df)):
    ticker = df.loc[i, 'Ticker Yahoo']
    try:
        sus = yf.Ticker(ticker).sustainability
        scores = sus.loc[['environmentScore','socialScore','governanceScore','totalEsg'],'Value']
        esg_scores.loc[i] = [ticker, scores[0], scores[1], scores[2], scores[3]]
    except:
        pass
esg_scores



 53%|█████▎    | 189/357 [35:36<37:34, 13.42s/it] 